# IIC-3641 GML UC

In [1]:
import torch
print(torch.__version__)

device = torch.device('cuda')

2.4.1+cu118


## Aquí leemos los datos, los cuales están en la carpeta data

In [2]:
import numpy as np
import sys
import pickle as pkl
import networkx as nx
from scipy.sparse import csr_matrix, dia_matrix, lil_matrix, eye, vstack, isspmatrix_coo, coo_matrix, diags, triu

def parse_index_file(filename):
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index

def load_data(dataset):
    # load the data: x, tx, allx, graph
    names = ['x', 'tx', 'allx', 'graph']
    objects = []
    for i in range(len(names)):
        with open("data/ind.{}.{}".format(dataset, names[i]), 'rb') as f:
            if sys.version_info > (3, 0):
                objects.append(pkl.load(f, encoding='latin1'))
            else:
                objects.append(pkl.load(f))
    x, tx, allx, graph = tuple(objects)
    test_idx_reorder = parse_index_file("data/ind.{}.test.index".format(dataset))
    test_idx_range = np.sort(test_idx_reorder)

    if dataset == 'citeseer':
        # Fix citeseer dataset (there are some isolated nodes in the graph)
        # Find isolated nodes, add them as zero-vecs into the right position
        test_idx_range_full = range(min(test_idx_reorder), max(test_idx_reorder)+1)
        tx_extended = lil_matrix((len(test_idx_range_full), x.shape[1]))
        tx_extended[test_idx_range-min(test_idx_range), :] = tx
        tx = tx_extended

    features = vstack((allx, tx)).tolil()
    features[test_idx_reorder, :] = features[test_idx_range, :]
    adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))

    return adj, features

## Aquí se definen funciones que son útiles para trabajar con los datos, por ejemplo preprocesar el grafo para crear la matriz de adyacencia

In [3]:
def sparse_to_tuple(sparse_mx):
    if not isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

def preprocess_graph(adj):
    adj = coo_matrix(adj)
    adj_ = adj + eye(adj.shape[0])
    rowsum = np.array(adj_.sum(1))
    degree_mat_inv_sqrt = diags(np.power(rowsum, -0.5).flatten())
    adj_normalized = adj_.dot(degree_mat_inv_sqrt).transpose().dot(degree_mat_inv_sqrt).tocoo()
    return sparse_to_tuple(adj_normalized)

def mask_test_edges(adj):
    # Function to build test set with 10% positive links
    # NOTE: Splits are randomized and results might slightly deviate from reported numbers in the paper.
    # TODO: Clean up.

    # Remove diagonal elements
    adj = adj - dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
    adj.eliminate_zeros()
    # Check that diag is zero:
    assert np.diag(adj.todense()).sum() == 0

    adj_triu = triu(adj)
    adj_tuple = sparse_to_tuple(adj_triu)
    edges = adj_tuple[0]
    edges_all = sparse_to_tuple(adj)[0]
    num_test = int(np.floor(edges.shape[0] / 10.))
    num_val = int(np.floor(edges.shape[0] / 20.))

    all_edge_idx = list(range(edges.shape[0]))
    np.random.shuffle(all_edge_idx)
    val_edge_idx = all_edge_idx[:num_val]
    test_edge_idx = all_edge_idx[num_val:(num_val + num_test)]
    test_edges = edges[test_edge_idx]
    val_edges = edges[val_edge_idx]
    train_edges = np.delete(edges, np.hstack([test_edge_idx, val_edge_idx]), axis=0)

    def ismember(a, b, tol=5):
        rows_close = np.all(np.round(a - b[:, None], tol) == 0, axis=-1)
        return np.any(rows_close)

    test_edges_false = []
    while len(test_edges_false) < len(test_edges):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], edges_all):
            continue
        if test_edges_false:
            if ismember([idx_j, idx_i], np.array(test_edges_false)):
                continue
            if ismember([idx_i, idx_j], np.array(test_edges_false)):
                continue
        test_edges_false.append([idx_i, idx_j])

    val_edges_false = []
    while len(val_edges_false) < len(val_edges):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], train_edges):
            continue
        if ismember([idx_j, idx_i], train_edges):
            continue
        if ismember([idx_i, idx_j], val_edges):
            continue
        if ismember([idx_j, idx_i], val_edges):
            continue
        if val_edges_false:
            if ismember([idx_j, idx_i], np.array(val_edges_false)):
                continue
            if ismember([idx_i, idx_j], np.array(val_edges_false)):
                continue
        val_edges_false.append([idx_i, idx_j])

    assert ~ismember(test_edges_false, edges_all)
    assert ~ismember(val_edges_false, edges_all)
    assert ~ismember(val_edges, train_edges)
    assert ~ismember(test_edges, train_edges)
    assert ~ismember(val_edges, test_edges)

    data = np.ones(train_edges.shape[0])

    # Re-build adj matrix
    adj_train = csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape=adj.shape)
    adj_train = adj_train + adj_train.T

    # NOTE: these edge lists only contain single direction of edge!
    return adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false

## Trabajamos con CORA, input_dim esta definido por la dimensionalidad de los datos de entrada

In [4]:
dataset = 'cora'
model = 'VGAE'

input_dim = 1433 
hidden1_dim = 32
hidden2_dim = 16
use_feature = True

num_epoch = 200
learning_rate = 0.01

## Aquí se define el VGAE, implementado sobre capas convolucionales.

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os

class VGAE(nn.Module):
    def __init__(self, adj):
        super(VGAE,self).__init__()
        self.base_gcn = GraphConvSparse(input_dim, hidden1_dim, adj)
        self.gcn_mean = GraphConvSparse(hidden1_dim, hidden2_dim, adj, activation=lambda x:x)
        self.gcn_logstddev = GraphConvSparse(hidden1_dim, hidden2_dim, adj, activation=lambda x:x)

    def encode(self, X):
        hidden = self.base_gcn(X)
        self.mean = self.gcn_mean(hidden)
        self.logstd = self.gcn_logstddev(hidden)
        gaussian_noise = torch.randn(X.size(0), hidden2_dim)
        sampled_z = gaussian_noise*torch.exp(self.logstd) + self.mean
        return sampled_z

    def forward(self, X):
        Z = self.encode(X)
        A_pred = dot_product_decode(Z)
        return A_pred

    
class GraphConvSparse(nn.Module):
    def __init__(self, input_dim, output_dim, adj, activation = F.relu, **kwargs):
        super(GraphConvSparse, self).__init__(**kwargs)
        self.weight = glorot_init(input_dim, output_dim) 
        self.adj = adj
        self.activation = activation

    def forward(self, inputs):
        x = inputs
        x = torch.mm(x,self.weight)
        x = torch.mm(self.adj, x)
        outputs = self.activation(x)
        return outputs


def dot_product_decode(Z):
    A_pred = torch.sigmoid(torch.matmul(Z,Z.t()))
    return A_pred

def glorot_init(input_dim, output_dim):
    init_range = np.sqrt(6.0/(input_dim + output_dim))
    initial = torch.rand(input_dim, output_dim)*2*init_range - init_range
    return nn.Parameter(initial)

## Aquí leemos y preprocesamos los datos

In [6]:
from torch.optim import Adam
from sklearn.metrics import roc_auc_score, average_precision_score
import time


# Train on CPU (hide GPU) due to memory constraints
#os.environ['CUDA_VISIBLE_DEVICES'] = ""

adj, features = load_data(dataset)

A = adj

# Store original adjacency matrix (without diagonal entries) for later
adj_orig = adj
adj_orig = adj_orig - dia_matrix((adj_orig.diagonal()[np.newaxis, :], [0]), shape=adj_orig.shape)
adj_orig.eliminate_zeros()

adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj)
adj = adj_train

# Some preprocessing
adj_norm = preprocess_graph(adj)


num_nodes = adj.shape[0]

features = sparse_to_tuple(features.tocoo())
num_features = features[2][1]
features_nonzero = features[1].shape[0]

/tmp/ipykernel_145710/276524816.py:20: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  objects.append(pkl.load(f, encoding='latin1'))


## Entran en la VGAE A y X. En el caso de A entra nomalizada.

In [7]:
# Create Model
pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)

adj_label = adj_train + eye(adj_train.shape[0])
adj_label = sparse_to_tuple(adj_label)

adj_norm = torch.sparse.FloatTensor(torch.LongTensor(adj_norm[0].T), 
                            torch.FloatTensor(adj_norm[1]), 
                            torch.Size(adj_norm[2]))
adj_label = torch.sparse.FloatTensor(torch.LongTensor(adj_label[0].T), 
                            torch.FloatTensor(adj_label[1]), 
                            torch.Size(adj_label[2]))
features = torch.sparse.FloatTensor(torch.LongTensor(features[0].T), 
                            torch.FloatTensor(features[1]), 
                            torch.Size(features[2]))

weight_mask = adj_label.to_dense().view(-1) == 1
weight_tensor = torch.ones(weight_mask.size(0)) 
weight_tensor[weight_mask] = pos_weight

/tmp/ipykernel_145710/503417153.py:8: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:641.)
  adj_norm = torch.sparse.FloatTensor(torch.LongTensor(adj_norm[0].T),


## La implementación pasa A al constructor. X entra en el forward

In [8]:
model = VGAE(adj_norm)

In [9]:
optimizer = Adam(model.parameters(), lr=learning_rate)

In [10]:
def get_scores(edges_pos, edges_neg, adj_rec):

    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    # Predict on test set of edges
    preds = []
    pos = []
    for e in edges_pos:
        # print(e)
        # print(adj_rec[e[0], e[1]])
        preds.append(sigmoid(adj_rec[e[0], e[1]].item()))
        pos.append(adj_orig[e[0], e[1]])

    preds_neg = []
    neg = []
    for e in edges_neg:

        preds_neg.append(sigmoid(adj_rec[e[0], e[1]].data))
        neg.append(adj_orig[e[0], e[1]])

    preds_all = np.hstack([preds, preds_neg])
    labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds_neg))])
    roc_score = roc_auc_score(labels_all, preds_all)
    ap_score = average_precision_score(labels_all, preds_all)

    return roc_score, ap_score


def get_acc(adj_rec, adj_label):
    labels_all = adj_label.to_dense().view(-1).long()
    preds_all = (adj_rec > 0.5).view(-1).long()
    accuracy = (preds_all == labels_all).sum().float() / labels_all.size(0)
    return accuracy

## El ciclo de entrenamiento usa la función de pérdida vista en clases

In [11]:
for epoch in range(num_epoch):
    t = time.time()

    A_pred = model(features)
    optimizer.zero_grad()
    loss = log_lik = norm*F.binary_cross_entropy(A_pred.view(-1), adj_label.to_dense().view(-1), weight = weight_tensor)
    kl_divergence = 0.5/ A_pred.size(0) * (1 + 2*model.logstd - model.mean**2 - torch.exp(model.logstd)**2).sum(1).mean()
    loss -= kl_divergence

    loss.backward()
    optimizer.step()

    train_acc = get_acc(A_pred,adj_label)

    val_roc, val_ap = get_scores(val_edges, val_edges_false, A_pred)
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(loss.item()),
          "train_acc=", "{:.5f}".format(train_acc), "val_roc=", "{:.5f}".format(val_roc),
          "val_ap=", "{:.5f}".format(val_ap),
          "time=", "{:.5f}".format(time.time() - t))


test_roc, test_ap = get_scores(test_edges, test_edges_false, A_pred)
print("End of training!", "test_roc=", "{:.5f}".format(test_roc),
      "test_ap=", "{:.5f}".format(test_ap))

Epoch: 0001 train_loss= 1.79959 train_acc= 0.49588 val_roc= 0.49217 val_ap= 0.53069 time= 0.13733
Epoch: 0002 train_loss= 1.47549 train_acc= 0.47979 val_roc= 0.49884 val_ap= 0.50028 time= 0.10144
Epoch: 0003 train_loss= 1.30103 train_acc= 0.44440 val_roc= 0.53796 val_ap= 0.51577 time= 0.10317
Epoch: 0004 train_loss= 1.11318 train_acc= 0.39691 val_roc= 0.53972 val_ap= 0.55298 time= 0.09967
Epoch: 0005 train_loss= 1.00111 train_acc= 0.35330 val_roc= 0.55126 val_ap= 0.52792 time= 0.09593
Epoch: 0006 train_loss= 0.89926 train_acc= 0.31061 val_roc= 0.58601 val_ap= 0.56507 time= 0.09684
Epoch: 0007 train_loss= 0.79474 train_acc= 0.31727 val_roc= 0.58888 val_ap= 0.57015 time= 0.09250
Epoch: 0008 train_loss= 0.74906 train_acc= 0.34623 val_roc= 0.60955 val_ap= 0.60040 time= 0.10069
Epoch: 0009 train_loss= 0.72238 train_acc= 0.36476 val_roc= 0.58130 val_ap= 0.55456 time= 0.10046
Epoch: 0010 train_loss= 0.71193 train_acc= 0.29284 val_roc= 0.59985 val_ap= 0.59670 time= 0.10168
Epoch: 0011 train_lo

Epoch: 0085 train_loss= 0.45721 train_acc= 0.54330 val_roc= 0.90935 val_ap= 0.90884 time= 0.11126
Epoch: 0086 train_loss= 0.45711 train_acc= 0.54260 val_roc= 0.90666 val_ap= 0.90415 time= 0.11248
Epoch: 0087 train_loss= 0.45692 train_acc= 0.54322 val_roc= 0.90768 val_ap= 0.90560 time= 0.10059
Epoch: 0088 train_loss= 0.45593 train_acc= 0.54248 val_roc= 0.90724 val_ap= 0.90657 time= 0.10583
Epoch: 0089 train_loss= 0.45592 train_acc= 0.54410 val_roc= 0.91239 val_ap= 0.90825 time= 0.11226
Epoch: 0090 train_loss= 0.45516 train_acc= 0.54516 val_roc= 0.90640 val_ap= 0.90729 time= 0.09353
Epoch: 0091 train_loss= 0.45448 train_acc= 0.54539 val_roc= 0.91021 val_ap= 0.90384 time= 0.10388
Epoch: 0092 train_loss= 0.45439 train_acc= 0.54502 val_roc= 0.91888 val_ap= 0.91664 time= 0.10172
Epoch: 0093 train_loss= 0.45397 train_acc= 0.54423 val_roc= 0.90311 val_ap= 0.90499 time= 0.09719
Epoch: 0094 train_loss= 0.45369 train_acc= 0.54449 val_roc= 0.89796 val_ap= 0.90182 time= 0.09381
Epoch: 0095 train_lo

Epoch: 0169 train_loss= 0.43951 train_acc= 0.54857 val_roc= 0.91386 val_ap= 0.91822 time= 0.10656
Epoch: 0170 train_loss= 0.43993 train_acc= 0.54899 val_roc= 0.91382 val_ap= 0.91719 time= 0.11092
Epoch: 0171 train_loss= 0.43902 train_acc= 0.54924 val_roc= 0.92207 val_ap= 0.92692 time= 0.10922
Epoch: 0172 train_loss= 0.43922 train_acc= 0.55023 val_roc= 0.92131 val_ap= 0.92589 time= 0.09458
Epoch: 0173 train_loss= 0.43885 train_acc= 0.54885 val_roc= 0.91048 val_ap= 0.91593 time= 0.10263
Epoch: 0174 train_loss= 0.43892 train_acc= 0.54999 val_roc= 0.91642 val_ap= 0.92464 time= 0.10531
Epoch: 0175 train_loss= 0.43882 train_acc= 0.55007 val_roc= 0.92008 val_ap= 0.92404 time= 0.10545
Epoch: 0176 train_loss= 0.43846 train_acc= 0.55079 val_roc= 0.91547 val_ap= 0.92022 time= 0.09676
Epoch: 0177 train_loss= 0.43855 train_acc= 0.54881 val_roc= 0.91404 val_ap= 0.91847 time= 0.11677
Epoch: 0178 train_loss= 0.43872 train_acc= 0.55133 val_roc= 0.90942 val_ap= 0.91527 time= 0.11012
Epoch: 0179 train_lo

## Vamos a explorar lo que resultó

In [12]:
Z = model.encode(features)

## Observemos que el VGAE no es probabilístico

In [ ]:
A_pred = torch.sigmoid(torch.matmul(Z,Z.t()))

In [14]:
type(A_pred)

torch.Tensor

In [15]:
A_pred.shape

torch.Size([2708, 2708])

## La A reconstruida la voy a pasar a densa para hacer la diferencia con la entrada

In [16]:
dense_A = torch.tensor(A.toarray())

In [17]:
dense_A.shape

torch.Size([2708, 2708])

In [18]:
l1_diff = torch.norm(A_pred - dense_A, p=1)
print(l1_diff)

tensor(3614781.2500, grad_fn=<LinalgVectorNormBackward0>)


In [19]:
A_pred

tensor([[0.9335, 0.5928, 0.7425,  ..., 0.5674, 0.9246, 0.8510],
        [0.5928, 0.9691, 0.9287,  ..., 0.2527, 0.7264, 0.5822],
        [0.7425, 0.9287, 0.9687,  ..., 0.4744, 0.9346, 0.8611],
        ...,
        [0.5674, 0.2527, 0.4744,  ..., 0.9393, 0.4420, 0.5902],
        [0.9246, 0.7264, 0.9346,  ..., 0.4420, 0.9897, 0.9622],
        [0.8510, 0.5822, 0.8611,  ..., 0.5902, 0.9622, 0.9553]],
       grad_fn=<SigmoidBackward0>)

In [20]:
dense_A

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 1, 0]])